# Data Testing For Obj Detection

In [2]:
# my tool box for pytorch
from p3self.matchbox import *
from constant import *
from utils import *

In [3]:
%ls {IMG}|wc -l

    5000


In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import matplotlib.text as text
import numpy as np
import pandas as pd
import json
from glob import glob
from PIL import Image
import os

os.system("mkdir -p /data/bbsample")

0

In [6]:
imgdf=pd.DataFrame(jsdict["images"])
ann_df=pd.DataFrame(jsdict["annotations"])
urls = glob(IMG+"/*")
rdimg = np.random.choice(urls)
def get_id(url):
    return int(url.split("/")[-1].split(".")[0])

In [10]:
def df_data(imgdf,ann_df,shuffle=True):
    """
    imgdf:
    A dataframe about images, fields: "id","file_name","height","width"
    ann_df:
    A dataframe about annotation, fields: "image_id","category_id","bbox",
    The field "bbox" is a list of 4 values: x,y,height, width of the bounding box
    """
    data_df=pd.merge(ann_df[["bbox","category_id","image_id"]],
                     imgdf[["id","file_name","height","width"]],
                     left_on="image_id",right_on="id")
    
    data_df["cate_id_oh"] = data_df["category_id"].apply(lambda x:idx2id[x])
    if shuffle:
        data_df = data_df.sample(frac=1).reset_index(drop=True)
    print("total data rows",len(data_df))
    return data_df

data_df = df_data(imgdf,ann_df)
data_df.head()

total data rows 36781


,bbox,category_id,image_id,id,file_name,height,width,cate_id_oh
0,"[434.0, 38.54, 21.59, 61.31]",44,84241,84241,000000084241.jpg,396,640,39
1,"[253.62, 369.16, 12.12, 35.55]",1,93717,93717,000000093717.jpg,427,640,0
2,"[375.0, 126.69, 122.75, 125.0]",67,435081,435081,000000435081.jpg,500,500,60
3,"[383.88, 147.23, 116.12, 85.56]",63,468501,468501,000000468501.jpg,375,500,57
4,"[259.95, 189.77, 239.19, 143.23]",63,293804,293804,000000293804.jpg,333,500,57


In [11]:
bbox_array = np.array(data_df.bbox.tolist())
wh_array = data_df[["width","height"]].as_matrix()

def re_calibrate(bbox_array,wh_array):
    """return the resized bbox array"""
    bb_resized = (bbox_array/np.concatenate([wh_array,wh_array],axis=-1)) *SIZE
    
    true_bb = bb_resized/SCALE
    # switch xy as left top conner to center point
    true_bb[...,:2]=true_bb[...,:2]+true_bb[...,2:]/2
    # Labels' Anchor positions on the grid map
    grid_bbxy = np.floor(true_bb[...,:2])
    return bb_resized,true_bb,grid_bbxy

def find_best_anchors(true_bbwh):
    iou_score = []
    for b in range(BOX):
        wh_anc = np.tile(ANC_ARR[b],[true_bbwh.shape[0],1])
        true_area = true_bbwh.prod(axis=-1)
        anc_area = wh_anc.prod(axis=-1)
    
        inter_area = np.min([wh_anc,true_bbwh],axis=0).prod(axis=-1)
    
        union_area = true_area + anc_area - inter_area
        iou_score.append(inter_area/union_area)
    best_anchor_idx = np.array(iou_score).T.argmax(axis=-1)
    return best_anchor_idx

bb_resized,true_bb,grid_bbxy = re_calibrate(bbox_array,wh_array)
true_bbxy,true_bbwh = true_bb[...,:2],true_bb[...,2:]
best_anchor_idx = find_best_anchors(true_bbwh)

In [32]:
min_lbl = SCALE * 0.001

data_df["true_bb_x"],data_df["true_bb_y"],data_df["true_bb_w"],data_df["true_bb_h"]=true_bb[:,0],true_bb[:,1],true_bb[:,2],true_bb[:,3]
data_df["true_grid_x"],data_df["true_grid_y"]=grid_bbxy[:,0],grid_bbxy[:,1]

# data_df["true_bb_x"]=data_df["true_bb_x"]-data_df["true_grid_x"]
# data_df["true_bb_y"]=data_df["true_bb_y"]-data_df["true_grid_y"]

data_df["best_anchor"]=best_anchor_idx
data_df_ = data_df[data_df["true_bb_w"]>min_lbl]
data_df_ = data_df_[data_df_["true_bb_h"]>min_lbl]


In [34]:
a=np.random.rand(2,3,4,5)
a[0,1,2]=np.ones(5)

In [27]:
from torch.utils.data import dataset

from constant import *
import numpy as np

from PIL import Image
from torchvision import transforms

class Data_Multi(dataset.Dataset):
    def __init__(self,data_df,testing=False,*args,**kwargs):
        """
        Object detection data generatorcd 
        """
        super(Data_Multi,self).__init__()
        
        for k,v in kwargs.items():
            setattr(self,k,v)
        self.data_df = data_df
        self.img_ids = list(set(list(data_df["image_id"])))
        self.testing=testing
        self.anc = np.tile(ANC_ARR[np.newaxis,np.newaxis,:,:],[FEAT_W,FEAT_H,1,1])
        
    def __len__(self):
        return len(self.ids)
        
    def __getitem__(self,idx):
        img=Image.open(self.id2url(self.image_ids[idx])).convert("RGB")

        sample = self.transform(img)
        
        original = self.trans_origin(img)
        
        img_df = self.data_df[self.data_df.image_id == self.image_ids[idx]].head(50)
        
        b_xywh = img_df[["true_bb_x","true_bb_y","true_bb_w","true_bb_h"]].as_matrix()
        posi = img_df[["true_grid_x","true_grid_y","best_anchor"]].as_matrix()
        t_xywh = self.b2t_xy(b_xywh)
        t_xywh = self.b2t_wh(t_xywh)
        
        N = t_xywh.shape[0]
        
        t_box = np.zeros((N,FEAT_W,FEAT_H,BOX,4))
        
        for i_lbl in xrange(N):
            t_box[i_lbl,posi[i_lbl,0],posi[i_lbl,1],posi[i_lbl,2]] = t_xywh[i_lbl]
            
        true_lbl,mask,vec_loc,t_xy,t_wh = self.true_label(self.true_adj_expand(self.true_adj[idx]),
                                                          self.vec_loc[idx])
        if self.testing:
            slicing = vec_loc[...,0],vec_loc[...,1],vec_loc[...,2]
            print("b",true_lbl[vec_loc[...,0],
                               vec_loc[...,1],
                               vec_loc[...,2],:5],
                  "\ttxy",t_xy[slicing],"\ttwh",t_wh[slicing])
        return sample,true_lbl,original,mask,vec_loc,t_xy,t_wh
    
    def get_id(self,url):
        return int(url.split("/")[-1].split(".")[0])
    
    def id2url(self,image_id):
        return IMG+self.data_df[self.data_df==image_id]["filename"][0]
    
    def true_label(self,true_adj,vec_loc):
        """
        Create true label for training
        """
        mask = self.mask_from_loca(vec_loc)
        true_adj_t = self.b2t_wh(self.b2t_xy(true_adj.copy()))
        true_adj *= np.tile(mask[:,:,:,np.newaxis],[1,1,1,true_adj.shape[-1]])
        
        true_adj_t *= np.tile(mask[:,:,:,np.newaxis],[1,1,1,true_adj_t.shape[-1]])
        t_xy,t_wh = true_adj_t[...,:2],true_adj_t[...,2:4]
        
        return true_adj,mask,vec_loc,t_xy,t_wh
    
    def b2t_xy(self,x):
        x[...,:2] = x[...,:2]-np.floor(x[...,:2])
        return x
    
    def b2t_wh(self,x):
        x[...,2:4] = np.clip(x[...,2:4],1e-2,12.999)
        x[...,2:4] = x[...,2:4]/self.anc
        x[...,2:4] = np.log(x[...,2:4])
        return x    
    
    def true_adj_expand(self,true_adj):
        return  np.tile(true_adj[np.newaxis,np.newaxis,np.newaxis,:],[FEAT_W,FEAT_H,BOX,1])
    
    def mask_from_loca(self,vec_loc):
        """
        return mask tensor [batch_size,grid_w,grid_h,box] according to grid location
        """
        mask=np.eye(BOX)[vec_loc[:,2:]]
        mask_w=np.eye(FEAT_W)[vec_loc[:,:1]].reshape(FEAT_H,1,1)
        mask_h=np.eye(FEAT_H)[vec_loc[:,1:2]].reshape((1,FEAT_H,1))
        
        mask = np.tile(mask,[FEAT_W,FEAT_H,1])
        mask *= np.tile(mask_w,[1,FEAT_H,BOX])
        mask *= np.tile(mask_h,[FEAT_W,1,BOX])
        return mask